In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

In [4]:
#! Set these variables
work_dir_name = "choroid_resegment1"

In [5]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
drive_root = Path("/media/smbshare")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"


Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [6]:
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels("choroid_t1_flair", suffix_list=["CH", "ED", "DT"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 305.79it/s]


In [7]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    print("Scan: ", scan.subid, scan.label_path)
    print("Scan: ", inference.subid, inference.label_path)

Scan:  1010 /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/choroid_t1_flair.nii.gz
Scan:  1010 /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz
Scan:  1033 /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/choroid_t1_flair.nii.gz
Scan:  1033 /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1033/ses-20171117/flair.t1_ensemble.nii.gz
Scan:  1152 /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/choroid_t1_flair.nii.gz
Scan:  1152 /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1152/ses-20170529/flair.t1_ensemble.nii.gz
Scan:  1163 /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/choroid_t1_flair.nii.gz
Scan:  1163 /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1163/ses-20180907/flair.t1_ensemble.nii.gz
Scan:  1548 /media/smbshare/3Tpioneer_bids/sub-ms1548/ses-202106

Calculate Dice Scores

In [8]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.label_path).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2)
    print("{}: {:0.2f}".format(scan.subid, score))

1010 1010
Dice Score: 0.7071336878782322
1033 1033
Dice Score: 0.7830172263789803
1152 1152
Dice Score: 0.7243532560214094
1163 1163
Dice Score: 0.7496891835888935
1548 1548
Dice Score: 0.77160696680907
2097 2097
Dice Score: 0.6735943588791984
2164 2164
Dice Score: 0.7819075712881023


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [ ]:
inference: Scan = ensemble_out_dataset[0]
label_path = inference.label_path
new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
utils.set_label_value(label_path, new_label_path, 2)

PosixPath('/home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble_val2.nii.gz')

In [10]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, "-s", manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1033/ses-20171117/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1152/ses-20170529/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/t1.nii.gz -s /home/s

Produce commands to open images and labels in freeview

In [1]:
image_paths = ["one", "twp", "three"]
step1 = " -g ".join(image_paths)
step1.split(" ")

['one', '-g', 'twp', '-g', 'three']

In [ ]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat